In [31]:
import warnings
warnings.filterwarnings("ignore")


In [32]:
import os
from dotenv import load_dotenv
load_dotenv()

serper_api_key =os.getenv("SERPER_API_KEY")

In [33]:
from crewai import Agent,Task,Crew,LLM

llm=LLM(
    model="ollama/gemma3:4b",
    base_url="http://localhost:11434"
 )

In [34]:
from crewai_tools import (
    FileReadTool,
    ScrapeWebsiteTool,
    MDXSearchTool,
    SerperDevTool
)
read_resume = FileReadTool('./fake_resume.md')
semantic_search_resume = MDXSearchTool(
    mdx='./fake_resume.md',
    config=dict(
        llm=dict(
            provider="ollama",
            config=dict(
                model="gemma2:2b",
            ),
        ),
        embedder=dict(
            provider="ollama",
            config=dict(
                model="mxbai-embed-large", 
            ),
        ),
    )
)


In [35]:

search_tool = SerperDevTool()
scrape_tool= ScrapeWebsiteTool()

### Creating Agent

In [36]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=llm
)

In [37]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm=llm
)

In [38]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=llm
)

In [39]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm=llm
)

### Creating Tasks

In [40]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [41]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [42]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [43]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


**Creating the Crew**

In [44]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],
    verbose=True,
    llm=llm
)

In [45]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e3bab327-b6a6-4e55-9dd3-288621057331                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Job Researcher                                                                                     │
│                                                                                                                 │
│  Task: Analyze the job posting URL provided                                                                     │
│  (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI  │
│  +Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and    │
│  identify and categorize the requirements.                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Personal Profiler for Engineers                                                                         │
│                                                                                                                 │
│  Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura)   │
│  URLs, and personal write-up (Noah is an accomplished Software                                                  │
│      Engineering Leader with 18 years of experience, specializing in                                            │
│      managing remote and in-office teams, and expert in multiple                                                │
│      programming languages and frameworks. He holds an MBA and a strong                                         │
│      background in AI and data science. Noah has successfully led                                               │
│      major tech initiatives and startups, proving his ability to drive                                          │
│      innovation and growth in the tech industry. Ideal for leadership                                           │
│      roles that require a strategic and innovative approach.). Utilize tools to extract and synthesize          │
│  information from these sources.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Job Researcher                                                                                     │
│                                                                                                                 │
│  Thought: Okay, I need to thoroughly analyze the job posting to extract the required skills, experiences, and   │
│  qualifications. I will use the "Read website content" tool to get the full content of the job posting and      │
│  then carefully examine it to identify the key requirements.                                                    │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"website_url\": \"https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&l  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: da27a575-9f65-4805-abaf-152d21f04b8f                                                                     │
│  Agent: Personal Profiler for Engineers                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resume Strategist for Engineers                                                                         │
│                                                                                                                 │
│  Task: Using the profile and job requirements obtained from previous tasks, tailor the resume to highlight the  │
│  most relevant areas. Employ tools to adjust and enhance the resume content. Make sure this is the best resume  │
│  even but don't make up any information. Update every section, inlcuding the initial summary, work experience,  │
│  skills, and education. All to better reflrect the candidates abilities and how it matches the job posting.     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resume Strategist for Engineers                                                                         │
│                                                                                                                 │
│  Thought: Okay, I need to revamp Noah Moura's resume to align with the AI Fund job requirements and the         │
│  information extracted about his skills and experience. I will focus on highlighting his relevant skills,       │
│  experience, and qualifications, emphasizing his adaptability, problem-solving abilities, and experience with   │
│  scalable AI systems. I will also adjust the language to reflect the innovative and ambitious nature of AI      │
│  Fund.                                                                                                          │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"AI Fund job description\"}"                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'AI Fund job description', 'type': 'search', 'num': 10, 'engine': 'google'},        │
│  'organic': [{'title': 'Careers - AI Fund', 'link': 'https://aifund.ai/careers/', 'snippet': 'Co-founding       │
│  companies together from idea to impact. AI Fund is a venture studio. We collaborate with entrepreneurs and     │
│  corporate partners to create great ...', 'position': 1}, {'title': 'AI Fund: Jobs | LinkedIn', 'link':         │
│  'https://www.linkedin.com/company/aifund/jobs', 'snippet': 'Jobs at AI Fund · Chief Technology Officer (Haven  │
│  Corp.)-Founding Team · Software Engineering Intern · Full-Stack Engineer Intern · Founder-in-Residence/CEO (   │
│  ...', 'position': 2}, {'title': 'Advisor, Venture Investing - AI Fund - Growth Equity Interview Guide',        │
│  'link': 'https://growthequityinterviewguide.com/job/advisor-venture-advisor-ai-fund', 'snippet': 'AI Fund is   │
│  looking for a Venture Advisor to help launch and grow AI-focused startups. This part-time role involves        │
│  working closely with teams to validate ideas.', 'position': 3}, {'title': 'Executive Assistant @ AI Fund in    │
│  Palo Alto, CA - John Gannon Blog', 'link':                                                                     │
│  'https://johngannonblog.com/job/executive-assistant-ai-fund-in-palo-alto-ca/', 'snippet': 'Specifically, you   │
│  will perform a wide variety of office/operations/admin tasks, including assisting with calendar management,    │
│  travel ...', 'position': 4}, {'title': 'Principal ~ AI Fund ~ Investing - Growth Equity Interview Guide',      │
│  'link': 'https://growthequityinterviewguide.com/job/principal-ai-fund-investing-ai-fund', 'snippet': 'AI Fund  │
│  is looking for a Principal to join their investment team in Palo Alto, CA. This role involves driving          │
│  projects from ideation to execution.', 'position': 5}, {'title': 'AI Fund hiring AI Engineer in Palo Alto, CA  │
│  | LinkedIn', 'link': 'https://www.linkedin.com/jobs/view/ai-engineer-at-ai-fund-4240709034', 'snippet':        │
│  'Seniority level. Mid-Senior level · Employment type. Full-time · Job function. Engineering · Industries.      │
│  Venture Capital and Private Equity P...                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

In [ ]:
display(Markdown("./interview_materials.md"))